Q1. Find the most popular Pinterest category people post to based on their country.

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

# join the tables on the ind column, groupby the country and category then finally count the distinct unique_id as category_count because there could be duplicates of the unique_idf
df = spark.table("global_temp.pin").join(spark.table("global_temp.geo"), "ind").groupBy("country", "category").agg(f.countDistinct("unique_id").alias("category_count"))

# create window partition by country and order it by the number in category count
window = Window.partitionBy("country").orderBy(f.desc("category_count"))
#apply the window function using dense_rank because it takes into account whether there are two category with same count
df_with_dense_rank = df.withColumn("dr", f.dense_rank().over(window))
#the highest category for each country will have the dr column equal to 1 therefore filter for that and orderBy desc category count
df_highest_per_year = df_with_dense_rank.filter(f.col("dr") == 1).orderBy(f.desc("category_count"))

display(df_highest_per_year.select("country","category", "category_count"))



country,category,category_count
Algeria,quotes,20
Albania,art,15
Afghanistan,education,9
Argentina,tattoos,9
American Samoa,tattoos,8
Aruba,art,7
Andorra,tattoos,7
Bangladesh,christmas,5
Bulgaria,christmas,5
Antarctica (the territory South of 60 deg S),tattoos,4


Q2. Find which was the most popular category each year


In [0]:

# Step 1: Join and group by category and year and count the unique posts and give it an alias
df = spark.table("global_temp.geo").join(spark.table("global_temp.pin"), "ind").groupBy('category', f.year("timestamp").alias("year")).agg(f.countDistinct("unique_id").alias("category_count"))

# Step 2: Create a window function and partition by year and orderby desc category count
window = Window.partitionBy("year").orderBy(f.desc("category_count"))

#again dense rank over the counts because there might be two or more category that have the same popularity that year
df_with_dense_rank = df.withColumn("dr", f.dense_rank().over(window))

#step 3: filter for only the highest category per year
df_highest_per_year = df_with_dense_rank.filter(f.col("dr") == 1).orderBy(f.desc("year"))

display(df_highest_per_year.select("category", "category_count", "year"))



category,category_count,year
christmas,25,2022
education,20,2021
mens-fashion,22,2020
art,21,2019
christmas,25,2018
mens-fashion,6,2017


Q3. Find the user with the most followers in each country

In [0]:
# Step 1: Join and group by category and year and count the unique posts and give it an alias
df = spark.table("global_temp.pin").join(spark.table("global_temp.geo"), "ind")

# Step 2: Partition by the country and orderby follower_count to the use in subsquent window function call
window = Window.partitionBy('geo.country').orderBy(f.desc('pin.follower_count'))
#opted for row number due to pin post data may contain different posts by the same user therefore dense would select that same user for as many posts as they have
df_with_rn = df.withColumn("rn", f.row_number().over(window))

# Step 3: now that the window function is finished we have to extract the top users for each country
df_country_top_users = df_with_rn.filter(f.col('rn') == 1).orderBy(f.desc('pin.follower_count'))

# Step 4: top users for each country
display(df_country_top_users.select('country', 'follower_count', 'poster_name'))

# Step 5: The country that has the top user
display(df_country_top_users.select(f.col("country"),f.col("follower_count")).orderBy(f.desc("follower_count")).limit(1))



country,follower_count,poster_name
Angola,8000000,Tastemade
American Samoa,8000000,Mamas Uncut
Azerbaijan,6000000,Style Me Pretty
Burkina Faso,6000000,Behance
Bouvet Island (Bouvetoya),5000000,POPSUGAR
Albania,5000000,The Minds Journal
United States Virgin Islands,4000000,"Ugly Duckling House | DIY, Home, Woodworking, Crafts & Recipes"
Bangladesh,4000000,Better Homes and Gardens
Afghanistan,3000000,9GAG
Botswana,3000000,OkChicas


country,follower_count
American Samoa,8000000


Q4. Find the most popular category for different age groups: \

    The following age groups: 
      18-24
      25-35
      36-50
      +50

In [0]:
df = spark.table('global_temp.user').join(spark.table('global_temp.pin'), 'ind')
#based on the age, created a new column age_groups that will split users into age groups
df = df.withColumn('age_groups', f.when(f.col('age').between(18, 24), '18-24')
                                  .when(f.col('age').between(25, 35), '25-35')
                                  .when(f.col('age').between(36, 50), '36-50')
                                  .when(f.col('age') > 50, '50+'))
#Groups the dataframe by age group and then category and finally agg the distinct category counts                                  
df = df.groupBy('age_groups', 'category').agg(f.countDistinct("unique_id").alias("category_count"))

#creates a window that partitions by age groups and orders by category count
window = Window.partitionBy('age_groups').orderBy(f.desc('category_count'))

#again dense rank because there could be two most popular categories
df_with_dr = df.withColumn("dr", f.dense_rank().over(window))

#finally filters on the dr column to provide the top category/categories for each age group
top_category_by_age_groups = df_with_dr.filter(f.col("dr") == 1).orderBy(f.desc("category_count"))
display(top_category_by_age_groups.select('category','category_count','age_groups'))


category,category_count,age_groups
tattoos,55,18-24
christmas,30,25-35
quotes,20,36-50
beauty,20,36-50
mens-fashion,10,50+


Q5. Find the median follower count for users of different age groups

In [0]:

df = spark.table('global_temp.user').join(spark.table('global_temp.pin'), 'ind')

#split based on age assign age groups
df = df.withColumn('age_groups', f.when(f.col('age').between(18, 24), '18-24')
                                  .when(f.col('age').between(25, 35), '25-35')
                                  .when(f.col('age').between(36, 50), '36-50')
                                  .when(f.col('age') > 50, '50+'))

#the median following of each age group calculated by percentile_approx, in later pyspark versions there is a median function
display(df.groupBy('age_groups').agg(f.percentile_approx("follower_count", 0.5).alias("median_follower_count")))

                                

age_groups,median_follower_count
50+,1000
36-50,7000
18-24,92000
25-35,23000


Q6. Find how many users have joined each year?

In [0]:
#the number of users joined every year
df_number_of_users_each_year = spark.table('global_temp.user').groupby(f.year('date_joined').alias('post_year')).agg(f.countDistinct('user_name').alias('number_of_users_joined'))
display(df_number_of_users_each_year)

post_year,number_of_users_joined
2015,213
2016,408
2017,157


Q7. Find the median follower count of users based on their joining year

In [0]:
 #median follower count based on join year
 df_median_followers_by_join_year = spark.table('global_temp.user').join(spark.table('global_temp.pin'), 'ind').groupby(f.year('date_joined').alias('post_year')).agg(f.percentile_approx("follower_count", 0.5).alias("median_follower_count"))

 display(df_median_followers_by_join_year)

post_year,median_follower_count
2015,117000
2016,21000
2017,2000


Q8. Find the median follower count of users  based on their age group and join year


In [0]:
df = spark.table('global_temp.user').join(spark.table('global_temp.pin'), 'ind')
#split into age groups based on age
df = df.withColumn('age_groups', f.when(f.col('age').between(18, 24), '18-24')
                                  .when(f.col('age').between(25, 35), '25-35')
                                  .when(f.col('age').between(36, 50), '36-50')
                                  .when(f.col('age') > 50, '50+'))
#median follower count based on age group further grouped by joining year
df = df.groupby('age_groups',f.year('date_joined').alias('post_year')).agg(f.percentile_approx("follower_count", 0.5).alias("median_follower_count"))
#ordered by age group and descending join year
display(df.select('age_groups', 'post_year', 'median_follower_count').orderBy('age_groups', f.desc('post_year')))



age_groups,post_year,median_follower_count
18-24,2017,11000
18-24,2016,42000
18-24,2015,221000
25-35,2017,1000
25-35,2016,24000
25-35,2015,37000
36-50,2017,3000
36-50,2016,9000
36-50,2015,10000
50+,2017,517
